# ADWORDS - Projeto 2 [by Query Explorer](https://ga-dev-tools.appspot.com/query-explorer)

IF697 — Introdução à Ciência dos Dados

Prof. Dr. Luciano Barbosa

Alunos: Hywre Cesar & Rafael Albuquerque

Utilizando o pre processado pelo projeto 1, iremos usar regressões para predizer os valores dos anúncios do nosso dataset inicial.

In [1]:
import pandas as pd
from sklearn import preprocessing
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import optuna
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import mlflow.sklearn
from sklearn.model_selection import cross_val_predict
plt.close("all")

## Importação do nosso dataset preprocessado

In [2]:
dataset = pd.read_csv("preprocessed_final_dataset.csv")
dataset.head()

,Unnamed: 0,dateHourMinute,adMatchedQuery,city,screenResolution,browser,operatingSystem,mobileDeviceInfo,adSlot,adMatchType,impressions,adClicks,adCost,pageviews,timeOnPage,ctr,screenResolutionDist,adMatchedQueryLenDist,hasFarmaOnQuery
0,0,2019-01-15 07:53:00,farmácia são paulo olinda estoque,Recife,360x640,Chrome,Android,Motorola Moto G (4) Moto G4,Google search: Top,Phrase match,1.0,1.0,0.27,1.0,288.2,1.0,x-small,short,True
1,1,2019-01-15 07:54:00,farmacia independente olinda,Olinda,360x640,Chrome,Android,Samsung SM-A520F Galaxy A5 (2017),Google search: Top,Broad match,1.0,1.0,0.27,3.0,15.0,1.0,x-small,short,True
2,2,2019-01-15 08:12:00,vick baby rub comprar,Olinda,360x640,Chrome,Android,Samsung SM-G531H Galaxy Grand Prime,Google search: Other,Broad match,1.0,1.0,0.23,NaN,276.0,1.0,x-small,short,False
3,3,2019-01-15 08:50:00,lacday similar,Recife,320x569,Chrome,Android,Samsung SM-G570M J5 Prime,Google search: Top,Phrase match,1.0,1.0,0.27,3.0,38.0,1.0,x-small,very short,False
4,4,2019-01-15 09:39:00,velunid generico onde comprar,Paulista,360x640,Chrome,Android,Motorola Moto G (4) Moto G4,Google search: Top,Broad match,1.0,1.0,0.27,3.0,13.0,1.0,x-small,short,False


Iremos dropar algumas colunas:

`Unnamed: 0`: essa coluna foi um ruido na escrita do dataset, são os indexes do pre processamento
`adMatchedQuery`: a partir dessa coluna criamos duas novas, `adMatchedQueryLen` e `hasFarmaOnQuery` e iremos usar essas duas para a nossa predição
`screenResolution`: iremos usar a coluna `screenResolutionDist`, que é uma discretização dessa coluna

In [3]:
columnsToRemove = ['Unnamed: 0', 'adMatchedQuery', 'screenResolution']
dataset = dataset.drop(columns=columnsToRemove)

In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16183 entries, 0 to 16182
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dateHourMinute         16183 non-null  object 
 1   city                   16183 non-null  object 
 2   browser                16183 non-null  object 
 3   operatingSystem        16183 non-null  object 
 4   mobileDeviceInfo       16183 non-null  object 
 5   adSlot                 16183 non-null  object 
 6   adMatchType            16183 non-null  object 
 7   impressions            14241 non-null  float64
 8   adClicks               14245 non-null  float64
 9   adCost                 14188 non-null  float64
 10  pageviews              16054 non-null  float64
 11  timeOnPage             16123 non-null  float64
 12  ctr                    13136 non-null  float64
 13  screenResolutionDist   16183 non-null  object 
 14  adMatchedQueryLenDist  16183 non-null  object 
 15  ha

Vamos converter os tipos objects para category, quebrar `dateHourMinute` em novas colunas `year`, `month`, `day` e `hour` e converter `hasFarmaOnQuery` to int64

In [5]:
columnsToBecameCategory = ['city', 'browser', 'operatingSystem', 'mobileDeviceInfo', 'adSlot', 'adMatchType', 'screenResolutionDist', 'adMatchedQueryLenDist']
for column in columnsToBecameCategory:
    dataset[column] = dataset[column].astype('category')

dataset['year'] = pd.to_datetime(dataset['dateHourMinute']).dt.year
dataset['month'] = pd.to_datetime(dataset['dateHourMinute']).dt.month
dataset['day'] = pd.to_datetime(dataset['dateHourMinute']).dt.day
dataset['hour'] = pd.to_datetime(dataset['dateHourMinute']).dt.hour
dataset = dataset.drop(columns=['dateHourMinute'])

dataset['hasFarmaOnQuery'] = dataset['hasFarmaOnQuery'].astype('int64')

Percebemos que temos alguma linhas com nan, não sabemos o motivo, pois esse dataset foi preprocessado no primeiro projeto e removemos nan. Vamos remover os NaN de maneira simples aqui.

In [6]:
dataset.dropna(inplace=True)

## Normalização de colunas numéricas

In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11335 entries, 0 to 16181
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   city                   11335 non-null  category
 1   browser                11335 non-null  category
 2   operatingSystem        11335 non-null  category
 3   mobileDeviceInfo       11335 non-null  category
 4   adSlot                 11335 non-null  category
 5   adMatchType            11335 non-null  category
 6   impressions            11335 non-null  float64 
 7   adClicks               11335 non-null  float64 
 8   adCost                 11335 non-null  float64 
 9   pageviews              11335 non-null  float64 
 10  timeOnPage             11335 non-null  float64 
 11  ctr                    11335 non-null  float64 
 12  screenResolutionDist   11335 non-null  category
 13  adMatchedQueryLenDist  11335 non-null  category
 14  hasFarmaOnQuery        11335 non-null 

In [8]:
columnsToNormalize = ['impressions', 'adClicks', 'pageviews', 'timeOnPage', 'ctr', 'hasFarmaOnQuery','year', 'month', 'day', 'hour']
dataframe_to_normalize = dataset[columnsToNormalize].values
min_max_scaler = preprocessing.MinMaxScaler()
dataset[columnsToNormalize] = pd.DataFrame(min_max_scaler.fit_transform(dataframe_to_normalize), columns=columnsToNormalize, index = dataset.index)

## Tratamento de features categoricas

In [9]:
dataset.describe(include=['category'])

,city,browser,operatingSystem,mobileDeviceInfo,adSlot,adMatchType,screenResolutionDist,adMatchedQueryLenDist
count,11335,11335,11335,11335,11335,11335,11335,11335
unique,14,10,4,583,4,3,5,8
top,Olinda,Chrome,Android,Apple iPhone,Google search: Top,Broad match,x-small,short
freq,4721,10242,10640,645,9501,8009,11183,5958


Para colunas categoricas ordinais, `screenResolutionDist` e `adMatchedQueryLenDist`, iremos fazer uma conversão simples de 1 até n-1 labels, mantendo a ordem.

Para colunas categoricas nominais, iremos usar a tecnica de BinaryEncoding. Essa tecnica é bem parecida com a One-Hot enconding, mas otimizando a quantidade de feature adicionais.

In [10]:
screenResolutionMap = {'x-small': 1, 'small': 2, 'medium': '3', 'large': 4, 'xlarge': 5, '2x-large': 6}
adMatchedQueryLenDistMap = {'very short': 1, 'short': 2, 'medium': 3, 'long': 4, 'very long': 5, 'x-long': 6, '2x-long': 7, '3x-long': 8}

dataset.screenResolutionDist.replace(screenResolutionMap, inplace=True)
dataset.adMatchedQueryLenDist.replace(adMatchedQueryLenDistMap, inplace=True)
dataset.screenResolutionDist = dataset.screenResolutionDist.astype('int64')
dataset.adMatchedQueryLenDist = dataset.adMatchedQueryLenDist.astype('int64')

In [11]:
columnsToApplyBinaryEncoder = ['city', 'browser', 'operatingSystem', 'mobileDeviceInfo', 'adSlot', 'adMatchType']
encoder = ce.BinaryEncoder(cols=columnsToApplyBinaryEncoder)
dataset = encoder.fit_transform(dataset)
dataset.head()

C:\Users\rafae\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


,city_0,city_1,city_2,city_3,city_4,browser_0,browser_1,browser_2,browser_3,browser_4,...,pageviews,timeOnPage,ctr,screenResolutionDist,adMatchedQueryLenDist,hasFarmaOnQuery,year,month,day,hour
0,0,0,0,0,1,0,0,0,0,1,...,0.000000,0.173058,0.0,1,2,1.0,0.0,0.0,0.466667,0.20
1,0,0,0,1,0,0,0,0,0,1,...,0.181818,0.007282,0.0,1,2,1.0,0.0,0.0,0.466667,0.20
3,0,0,0,0,1,0,0,0,0,1,...,0.181818,0.021238,0.0,1,1,0.0,0.0,0.0,0.466667,0.25
4,0,0,0,1,1,0,0,0,0,1,...,0.181818,0.006068,0.0,1,2,0.0,0.0,0.0,0.466667,0.30
5,0,0,0,0,1,0,0,0,0,1,...,0.090909,0.173544,0.0,1,1,0.0,0.0,0.0,0.466667,0.30


Vamos normalizar as categorias nominais encodadas

In [12]:
columnsToNormalize = ['screenResolutionDist', 'adMatchedQueryLenDist']
dataframe_to_normalize = dataset[columnsToNormalize].values
dataset[columnsToNormalize] = pd.DataFrame(min_max_scaler.fit_transform(dataframe_to_normalize), columns=columnsToNormalize, index = dataset.index)

## Separação do Dataset em Treino, Validação e Teste

Vamos separar o dataset em 70% para treino e validação e 30% para teste

In [13]:
def split_data(X, y):
    return train_test_split(X, y, test_size=0.3, random_state=0)
    
X_train, X_test, y_train, y_test = split_data(dataset.drop('adCost', axis=1), dataset.adCost)

## Seleção de algoritmos para regressão

Iremos utilizar 7 algoritmos de ML, com parametros default, que aparecem em lista de mais utilizados e iremos avaliar-los usando as métricas r2 e mse para escolher os 4 melhores sobre o conjunto de teste

In [14]:
linearRegression = LinearRegression()
randomForest = RandomForestRegressor(random_state=0)
knn = KNeighborsRegressor()
mlp = MLPRegressor(random_state=0)
svr = SVR()
decisionTree = DecisionTreeRegressor(random_state=0)
gradientBoosting = GradientBoostingRegressor(random_state=0)

linearRegression.fit(X_train, y_train)
randomForest.fit(X_train, y_train)
knn.fit(X_train, y_train)
mlp.fit(X_train, y_train)
svr.fit(X_train, y_train)
decisionTree.fit(X_train, y_train)
gradientBoosting.fit(X_train, y_train)

linearRegressionPredictions = linearRegression.predict(X_test)
randomForestPredictions = randomForest.predict(X_test)
knnPredictions = knn.predict(X_test)
mlpPredictions = mlp.predict(X_test)
svrPredictions = svr.predict(X_test)
decisionTreePredictions = decisionTree.predict(X_test)
gradientBoostingPredictions = gradientBoosting.predict(X_test)

def getMetrics(name, predictions, shouldPrint = True):
    r2 = r2_score(y_test, predictions)
    mse = mean_squared_error(y_test, predictions)
    if(shouldPrint):
        print(name + ' metrics')
        print('R2', r2)
        print('MSE', mse)
        print()
    return r2, mse
    

getMetrics('linear regression', linearRegressionPredictions)
getMetrics('random forest', randomForestPredictions)
getMetrics('knn', knnPredictions)
getMetrics('mlp', mlpPredictions)
getMetrics('svm', svrPredictions)
getMetrics('decision tree', decisionTreePredictions)
getMetrics('gradient boosting', gradientBoostingPredictions)

linear regression metrics
R2 0.24369563825534657
MSE 0.004411630558766554

random forest metrics
R2 0.5721656081988754
MSE 0.0024956186588784893

knn metrics
R2 0.14992223450203734
MSE 0.004958624116549744

mlp metrics
R2 0.22905638023372388
MSE 0.004497023426126036

svm metrics
R2 0.17152099874588345
MSE 0.004832635462788791

decision tree metrics
R2 0.2142870048702218
MSE 0.0045831752863866515

gradient boosting metrics
R2 0.537651876186567
MSE 0.002696942150510006



(0.537651876186567, 0.002696942150510006)

Os 4 algoritmos que apresentaram a melhor performance foram: RandomForest, GradientBoosting, MLP e KNN. Agora iremos tentar melhorar-los ajustando hiperparametros.

## Seleção de features

Nosso dataset ficou com 42 features, queremos reduzir esse número para evitar `curse of dimensionality` e para ganhar tempo de execução nos nossos treinamentos. Iremos utilizar o `SelectKBest` do proprio sklearn e vamos varias o numero de features entre 1 até 42 e achar um numero que as nossas métricas dos classificadores escolhidos se mantenham.

In [ ]:
randomForestMetrics = []
gradiendBoostingMetrics = []
mlpMetrics = []
knnMetrics = []
X = dataset.drop('adCost', axis=1)
y = dataset.adCost
for i in range(1, len(X.columns)):
    X_new = SelectKBest(mutual_info_regression, k=i).fit_transform(X, y)
    X_train, X_test, y_train, y_test = split_data(X_new, y)
    randomForest = RandomForestRegressor(random_state=0)
    knn = KNeighborsRegressor()
    mlp = MLPRegressor(random_state=0)
    gradientBoosting = GradientBoostingRegressor(random_state=0)
    
    randomForest.fit(X_train, y_train)
    knn.fit(X_train, y_train)
    mlp.fit(X_train, y_train)
    gradientBoosting.fit(X_train, y_train)

    randomForestPredictions = randomForest.predict(X_test)
    knnPredictions = knn.predict(X_test)
    mlpPredictions = mlp.predict(X_test)
    gradientBoostingPredictions = gradientBoosting.predict(X_test)
    
    randomForestMetrics += [getMetrics('random forest', randomForestPredictions, False)]
    knnMetrics += [getMetrics('knn', knnPredictions, False)]
    mlpMetrics += [getMetrics('mlp', mlpPredictions, False)]
    gradiendBoostingMetrics += [getMetrics('gradient boosting', gradientBoostingPredictions, False)]

In [ ]:
df1 = pd.DataFrame(randomForestMetrics, columns=["random forest r2", "random forest mse"])
df2 = pd.DataFrame(gradiendBoostingMetrics, columns=["gradient boosting r2", "gradient boosting mse"])
df3 = pd.DataFrame(mlpMetrics, columns=["mlp r2", "mlp mse"])
df4 = pd.DataFrame(knnMetrics, columns=["knn r2", "knn mse"])

g = sns.lineplot(data=pd.concat([df1,df2, df3, df4], axis=1))
g.set_xticks(range(0, 41))
print()

De acordo com o gráfico a acima iremos usar as 7 melhores features para treinar nosso modelos

In [ ]:
X_new = SelectKBest(mutual_info_regression, k=7).fit_transform(X, y)
X_train, X_test, y_train, y_test = split_data(X_new, y)

## Seleção de hiperparametros

Iremos usar o framework Optuna para estimar os melhores hiperparametro rodando nosso modelos no conjunto de validação usando cross validation. As métricas que iremos utilizar para avaliar os modelos no conjunto de validação serão: r2 e MSE

In [ ]:
def optimizeRandomForest(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    model = RandomForestRegressor(n_estimators=n_estimators,
                                max_depth=max_depth, n_jobs=5, random_state=0)
    predicts = cross_val_predict(model, X_train, y_train)
    r2 = r2_score(y_train, predicts)
    mse = mean_squared_error(y_train, predicts)
    mlflow.start_run()
    mlflow.log_params(trial.params)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("mean_squared_error", mse)
    mlflow.sklearn.log_model(model, "random-forest")
    mlflow.end_run()
    return r2

def optimizeGradientBoosting(trial):
    learning_rate = trial.suggest_uniform('learning_rate', 0.001, 0.4)
    n_estimators = trial.suggest_int('n_estimators', 10, 100)
    max_depth = trial.suggest_int('max_depth', 2, 10)
    model = GradientBoostingRegressor(learning_rate=learning_rate,
                                n_estimators=n_estimators,
                                max_depth=max_depth, random_state=0)
    predicts = cross_val_predict(model, X_train, y_train)
    r2 = r2_score(y_train, predicts)
    mse = mean_squared_error(y_train, predicts)
    mlflow.start_run()
    mlflow.log_params(trial.params)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("mean_squared_error", mse)
    mlflow.sklearn.log_model(model, "gradient-boosting")
    mlflow.end_run()
    return r2

def optimizeMlp(trial):
    activation = trial.suggest_categorical('activation', ['identity', 'logistic', 'tanh', 'relu'])
    n_layers = trial.suggest_int('n_layers', 1, 8)
    layers = []
    for i in range(n_layers):
        layers.append(trial.suggest_int('layer ' + str(i), 10, 100))
    model = MLPRegressor(hidden_layer_sizes=tuple(layers),
                                activation=activation, random_state=0)
    predicts = cross_val_predict(model, X_train, y_train)
    r2 = r2_score(y_train, predicts)
    mse = mean_squared_error(y_train, predicts)
    mlflow.start_run()
    mlflow.log_params(trial.params)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("mean_squared_error", mse)
    mlflow.sklearn.log_model(model, "mlp")
    mlflow.end_run()
    return r2

def optimizeKnn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 50)
    model = KNeighborsRegressor(n_neighbors=n_neighbors)
    predicts = cross_val_predict(model, X_train, y_train)
    r2 = r2_score(y_train, predicts)
    mse = mean_squared_error(y_train, predicts)
    mlflow.start_run()
    mlflow.log_params(trial.params)
    mlflow.log_metric("r2_score", r2)
    mlflow.log_metric("mean_squared_error", mse)
    mlflow.sklearn.log_model(model, "knn")
    mlflow.end_run()
    return r2

n_trials=10

randomForestStudy = optuna.create_study(direction='maximize')
randomForestStudy.optimize(optimizeRandomForest, n_trials=n_trials)

gradientBoostingStudy = optuna.create_study(direction='maximize')
gradientBoostingStudy.optimize(optimizeGradientBoosting, n_trials=n_trials)

mlpStudy = optuna.create_study(direction='maximize')
mlpStudy.optimize(optimizeMlp, n_trials=n_trials)

knnStudy = optuna.create_study(direction='maximize')
knnStudy.optimize(optimizeKnn, n_trials=n_trials)

## Avaliando o modelo com o melhor modelo